# imports

In [ ]:
import ipywidgets as widgets
import pandas as pd
import os
import glob
from tqdm import tqdm
from IPython.display import HTML
from IPython.display import display


showB = HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
showB

# Metadata

In [2]:
country = "Colombia" # @param {type:"string"}
temporal_range = "2018-2020" # @param {type:"string"}
pais =widgets.Combobox(
    # value='John',
    placeholder='Escoja el pais',
    options=['Colombia', 'Brasil', 'Peru'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

tiempo = widgets.Text(
    value='2000-2010',
    placeholder='Type something',
    description='Temporal range:',
    disabled=False
)
tab_contents = ['Pais', 'Rango']
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, "Pais")
tab.set_title(1, "Rango temporal")
tab

# Load DataBase 1

In [3]:
ola = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
dataframe_1 = pd.read_csv("Casos_positivos_de_COVID-19_en_Colombia._20240519_datosabiertos.csv", low_memory=False)
ola

FileUpload(value={}, description='Upload')

## Filtrar Columnas vacias: Escoger umbral

In [4]:
def filter_columns_by_nan_threshold(dataframe, threshold):
    """
    Filters the columns of a dataframe based on the percentage of NaN values.

    Parameters:
    dataframe (pd.DataFrame): The input dataframe.
    threshold (float): The threshold for the percentage of NaN values. Columns with NaN percentages lower than this threshold will be included in the output.

    Returns:
    pd.DataFrame: A dataframe containing only the columns that have a lower percentage of NaN values than the threshold.
    """
    nan_percentage = dataframe.isna().mean() * 100  # Calculate percentage of NaNs in each column
    filtered_columns = nan_percentage[nan_percentage < threshold].index  # Get columns with NaN percentage below the threshold
    filtered_dataframe = dataframe[filtered_columns]  # Filter the dataframe
    return filtered_dataframe

umbral = widgets.IntSlider(
    value=10,
    min=0,
    max=100,
    step=10,
    description='Umbral:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
umbral

IntSlider(value=10, continuous_update=False, description='Umbral:', step=10)

## Filtrar Columnas vacias: Ejecutar filtrado

In [5]:
filtered_df = filter_columns_by_nan_threshold(dataframe_1, umbral.value)
filtro = len(dataframe_1.columns) - len(filtered_df.columns)
print(f"Se han filtrado {filtro} columnas que estaban {100-umbral.value}% vacias")

Se han filtrado 4 columnas que estaban 90% vacias


## Seleccionar columnas de interes

In [6]:
# Get the list of columns from the first dataframe
columns = list(filtered_df.columns)

# Create a checkbox for each option
checkboxes = [widgets.Checkbox(value=False, description=option) for option in columns]

# Create a "Select/Deselect All" checkbox
select_all = widgets.Checkbox(value=False, description="Select/Deselect All")

def select_all_changed(change):
    # Update all checkboxes based on the "Select/Deselect All" checkbox
    for checkbox in checkboxes:
        checkbox.value = change['new']

# Watch for changes on the "Select/Deselect All" checkbox
select_all.observe(select_all_changed, names='value')

# Display the "Select/Deselect All" checkbox and all other checkboxes
display(select_all)
display(widgets.VBox(checkboxes))

Checkbox(value=False, description='Select/Deselect All')

## Proceso de agregacion: Escoger funciones de agregacion

In [7]:
selected_columns_1 = [checkbox.description for checkbox in checkboxes if checkbox.value]

def mode(series):
    mode = series.mode()
    if len(mode) > 1:
        return ','.join(mode)
    else:
        return mode.iloc[0]

funciones = [mode, ['mean', 'std', 'min', 'max']]
tipo = ["Categorica", "Numerica"]
agg_d = {}
for column in selected_columns_1:
    agg_d[column] = widgets.Dropdown(
      options=tipo,
      value=tipo[0], # Default value
      description=f'{column}:',
  )
    display(agg_d[column])

Dropdown(description='Código DIVIPOLA departamento:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Nombre departamento:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Código DIVIPOLA municipio:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Nombre municipio:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Edad:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Sexo:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Tipo de contagio:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Estado:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Tipo de recuperación:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='Pertenencia étnica:', options=('Categorica', 'Numerica'), value='Categorica')

## Proceso de agregacion: Ejecutar agregacion 

In [8]:
agg = {}
for k, v in agg_d.items():
    if v == "categorica":
        agg[k] = funciones[0]
    else:
        agg[k] = funciones[0]
agg_df = filtered_df.groupby(["Código DIVIPOLA departamento"]).agg(agg)
agg_df["ID_DPTO"] = agg_df["Código DIVIPOLA departamento"]
agg_df = agg_df.loc[:,["ID_DPTO","Nombre departamento","Código DIVIPOLA municipio","Nombre municipio","Edad","Sexo","Tipo de contagio","Estado"]]
agg_df

,ID_DPTO,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Sexo,Tipo de contagio,Estado
Código DIVIPOLA departamento,,,,,,,,
5,5,ANTIOQUIA,5001,MEDELLIN,28,F,Comunitaria,Leve
8,8,ATLANTICO,8758,SOLEDAD,30,F,Comunitaria,Leve
11,11,BOGOTA,11001,BOGOTA,30,F,Comunitaria,Leve
13,13,BOLIVAR,13430,MAGANGUE,28,F,Comunitaria,Leve
15,15,BOYACA,15001,TUNJA,24,F,Relacionado,Leve
17,17,CALDAS,17001,MANIZALES,28,F,Comunitaria,Leve
18,18,CAQUETA,18001,FLORENCIA,26,M,Comunitaria,Leve
19,19,CAUCA,19001,POPAYAN,30,F,Comunitaria,Leve
20,20,CESAR,20001,VALLEDUPAR,30,F,Comunitaria,Leve


# Load DataBase 2

In [9]:
def preparar_datos_censo():
    file_pattern = os.path.join("data\input",'CNPV2018_5PER_A2_*')
    filtered_files = glob.glob(file_pattern)

    dfs = []
    c = 1
    for file in tqdm(filtered_files):
        df = pd.read_csv(file).loc[:,["U_DPTO", "U_MPIO", "P_SEXO", "P_EDADR", "P_ENFERMO", "PA_LO_ATENDIERON", "PA1_CALIDAD_SERV", "P_TRABAJO"]]
        dfs.append(df)
        c+=1
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

dataframes_2 = preparar_datos_censo()

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [05:16<00:00,  9.58s/it]


## Filtrar Columnas vacias: Escoger umbral

In [10]:
umbral2 = widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=10,
    description='Umbral:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
umbral2

IntSlider(value=30, continuous_update=False, description='Umbral:', step=10)

## Filtrar Columnas vacias: Ejecutar filtrado

In [11]:
filtered_df_2 = filter_columns_by_nan_threshold(dataframes_2, umbral2.value)
filtro_2 = len(dataframes_2.columns) - len(filtered_df_2.columns)
print(f"Se han filtrado {filtro_2} columnas que estaban {100-umbral2.value}% vacias")

Se han filtrado 2 columnas que estaban 70% vacias


## Seleccionar columnas de interes

In [12]:
# Get the list of columns from the first dataframe
columns2 = list(filtered_df_2.columns)

# Create a checkbox for each option
checkboxes_2 = [widgets.Checkbox(value=False, description=option) for option in columns2]

# Create a "Select/Deselect All" checkbox
select_all2 = widgets.Checkbox(value=False, description="Select/Deselect All")

def select_all_changed2(change):
    # Update all checkboxes based on the "Select/Deselect All" checkbox
    for checkbox in checkboxes_2:
        checkbox.value = change['new']

# Watch for changes on the "Select/Deselect All" checkbox
select_all2.observe(select_all_changed2, names='value')

# Display the "Select/Deselect All" checkbox and all other checkboxes
display(select_all2)
display(widgets.VBox(checkboxes_2))

Checkbox(value=False, description='Select/Deselect All')

## Proceso de agregacion: Escoger funciones de agregacion

In [13]:
selected_columns_2 = [checkbox.description for checkbox in checkboxes_2 if checkbox.value]
agg_d2 = {}
for column in selected_columns_2:
    agg_d2[column] = widgets.Dropdown(
      options=tipo,
      value=tipo[0], # Default value
      description=f'{column}:',
  )
    display(agg_d2[column])

Dropdown(description='U_DPTO:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='U_MPIO:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='P_SEXO:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='P_EDADR:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='P_ENFERMO:', options=('Categorica', 'Numerica'), value='Categorica')

Dropdown(description='P_TRABAJO:', options=('Categorica', 'Numerica'), value='Categorica')

## Proceso de agregacion. Ejecutar agregacion

In [14]:
agg2 = {}
for k, v in agg_d2.items():
    if v == "categorica":
        agg2[k] = funciones[0]
    else:
        agg2[k] = funciones[0]
agg_df2 = filtered_df_2.groupby(["U_DPTO"]).agg(agg2)
agg_df2 = agg_df2.loc[:,["U_MPIO","P_SEXO","P_EDADR","P_ENFERMO","P_TRABAJO"]]
agg_df2

,U_MPIO,P_SEXO,P_EDADR,P_ENFERMO,P_TRABAJO
U_DPTO,,,,,
5,1,2,5,2.0,1.0
8,1,2,4,2.0,1.0
11,1,2,5,2.0,1.0
13,1,2,3,2.0,1.0
15,1,2,4,2.0,1.0
17,1,2,5,2.0,1.0
18,1,1,3,2.0,1.0
19,1,2,4,2.0,1.0
20,1,2,3,2.0,1.0


# Ejecutar union

In [15]:
#@title Join Tables
merged_df = pd.merge(agg_df, agg_df2, left_on='Código DIVIPOLA departamento', right_on='U_DPTO', how='inner')
merged_df

,ID_DPTO,Nombre departamento,Código DIVIPOLA municipio,Nombre municipio,Edad,Sexo,Tipo de contagio,Estado,U_MPIO,P_SEXO,P_EDADR,P_ENFERMO,P_TRABAJO
0,5,ANTIOQUIA,5001,MEDELLIN,28,F,Comunitaria,Leve,1,2,5,2.0,1.0
1,8,ATLANTICO,8758,SOLEDAD,30,F,Comunitaria,Leve,1,2,4,2.0,1.0
2,11,BOGOTA,11001,BOGOTA,30,F,Comunitaria,Leve,1,2,5,2.0,1.0
3,13,BOLIVAR,13430,MAGANGUE,28,F,Comunitaria,Leve,1,2,3,2.0,1.0
4,15,BOYACA,15001,TUNJA,24,F,Relacionado,Leve,1,2,4,2.0,1.0
5,17,CALDAS,17001,MANIZALES,28,F,Comunitaria,Leve,1,2,5,2.0,1.0
6,18,CAQUETA,18001,FLORENCIA,26,M,Comunitaria,Leve,1,1,3,2.0,1.0
7,19,CAUCA,19001,POPAYAN,30,F,Comunitaria,Leve,1,2,4,2.0,1.0
8,20,CESAR,20001,VALLEDUPAR,30,F,Comunitaria,Leve,1,2,3,2.0,1.0
9,23,CORDOBA,23001,MONTERIA,31,F,Comunitaria,Leve,1,2,3,2.0,1.0
